In [1]:
!pip install -qU crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29

In [2]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [3]:
import os
from utils import get_bedrock_client, get_haiku_config, get_sonnet_config
from langchain_community.chat_models import BedrockChat  # Changed this import

In [12]:
bedrock_client = get_bedrock_client()
sonnet_config = get_sonnet_config()  # Using Sonnet config

# Create BedrockChat instance for CrewAI
llm = BedrockChat(
    client=bedrock_client,
    model_id="anthropic.claude-3-sonnet-20240229-v1:0",  # Updated model ID
    streaming=True,
    model_kwargs={
        "temperature": sonnet_config["temperature"],
        "top_p": sonnet_config["top_p"],
        "max_tokens": sonnet_config["max_tokens"],
        "anthropic_version": "bedrock-2023-05-31"
    }
)

# Set model name in environment if needed
os.environ["MODEL_NAME"] = "anthropic.claude-3-sonnet-20240229-v1:0"

Let us create get_completion, which is a helper function that sends a prompt to Claude and returns Claude's generated response. Run that cell now.

In [13]:
from botocore.exceptions import ClientError  # Import ClientError for exception handling

def get_completion(prompt, system_prompt=None):
    """
    Helper function to get a completion from the Bedrock client.
    
    Args:
        prompt (str): The user prompt to provide to the model.
        system_prompt (str, optional): An optional system prompt to set context for the model.

    Returns:
        str: The generated response from the model.
    """
    # Use existing configurations for inference and model parameters
    inference_config = {
        "temperature": 0.0,  # Minimal randomness
        "maxTokens": 200  # Maximum token limit
    }
    additional_model_fields = {
        "top_p": 1  # Full probability distribution
    }

    # Build the message structure, including system prompt in the user prompt context
    messages = [{"role": "user", "content": [{"text": f"{system_prompt}\n{prompt}" if system_prompt else prompt}]}]

    # Set up the Bedrock client request
    converse_api_params = {
        "modelId": sonnet_config["model"],  # Use the model ID from sonnet_config
        "messages": messages,
        "inferenceConfig": inference_config,
        "additionalModelRequestFields": additional_model_fields
    }

    # Make the API call
    try:
        response = bedrock_client.converse(**converse_api_params)
        # Extract and return the generated content
        return response['output']['message']['content'][0]['text']

    except ClientError as err:
        print(f"Client error occurred: {err.response['Error']['Message']}")
        return None

    except Exception as e:
        print(f"An unexpected error occurred: {str(e)}")
        return None


In [14]:
response = get_completion(
    prompt="Explain the significance of AI in modern business.",
    system_prompt="You are an expert in AI and its applications in business."
)
print(response)


Artificial Intelligence (AI) has become a game-changer in the modern business landscape, revolutionizing various aspects of operations and decision-making processes. The significance of AI in modern business can be highlighted through the following points:

1. Automation and Efficiency: AI enables businesses to automate repetitive and time-consuming tasks, leading to increased efficiency, reduced costs, and improved productivity. From automating customer service through chatbots to streamlining supply chain management, AI can optimize processes and free up human resources for more strategic and creative endeavors.

2. Data Analysis and Insights: AI excels at processing and analyzing vast amounts of data, enabling businesses to uncover valuable insights and patterns that would be difficult or impossible for humans to detect. This data-driven decision-making can lead to better strategic planning, targeted marketing campaigns, and personalized customer experiences.

3. Predictive Analytic

In [15]:
# Prompt
prompt = "Hello, Claude!"

# Get Claude's response
print(get_completion(prompt))

Hello! It's nice to meet you. How can I assist you today?
